<a href="https://colab.research.google.com/github/HosnawHb/Blood_Glucose_Prediction/blob/main/MOE_22July.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers import Input
from keras.layers import SimpleRNN
from keras.layers import Dense, Dropout
from keras.layers import LSTM, BatchNormalization, Bidirectional,GRU
from keras.layers import concatenate
from keras import Model
from keras.models import Sequential
import datetime
from keras.callbacks import ModelCheckpoint
import csv
import pickle
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import Normalizer

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/dataset/CGM/XPH5", 'rb') as f:
    X = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/dataset/CGM/yPH5", 'rb') as f:
    y = pickle.load(f)

In [ ]:
X = np.reshape(X, (len(X), len(X[1])))
y = np.reshape(y, (len(y), 1))

In [ ]:
X.shape, y.shape

((118186, 5), (118186, 1))

In [ ]:
Normalizer().fit_transform(X, y)

array([[0.51484397, 0.51220375, 0.49768251, 0.3313483 , 0.33926898],
       [0.55729412, 0.54149455, 0.36051759, 0.36913554, 0.36051759],
       [0.60119946, 0.40026807, 0.40983624, 0.40026807, 0.38751053],
       ...,
       [0.42280993, 0.44395043, 0.45099726, 0.4533462 , 0.46391645],
       [0.43469473, 0.44159465, 0.44389462, 0.45424449, 0.46114441],
       [0.4364541 , 0.4387273 , 0.4489567 , 0.45577629, 0.45577629]])

In [ ]:
"""
def splitter(X,y,testPercent):

  #Split train and test data

  X_train,X_test,y_train,y_test = train_test_split(X, y, test_size= testPercent)
  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)
  return X_train, X_test, y_train, y_test,X_valid, y_valid
"""

'\ndef splitter(X,y,testPercent):\n\n  #Split train and test data\n  \n  X_train,X_test,y_train,y_test = train_test_split(X, y, test_size= testPercent)\n  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)\n  return X_train, X_test, y_train, y_test,X_valid, y_valid\n'

In [ ]:
#X_train,X_test,y_train,y_test,X_valid,y_valid = splitter(X,y,0.1)

In [ ]:
X = np.reshape(X, (len(X), len(X[1])))
y = np.reshape(y, (len(y), 1))

In [ ]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)
X = X.to(torch.float32)
y = y.to(torch.float32)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size= 0.2)

In [ ]:
train_dataLoader = DataLoader([ [X_train[i], y_train[i]] for i in range(len(y_train))], shuffle=True, batch_size=32)
test_dataLoader = DataLoader([ [X_test[i], y_test[i]] for i in range(len(y_test))], shuffle=True, batch_size=32)

In [ ]:
train_dataLoader

In [ ]:
class MixtureOfExpertsBard(nn.Module):
    def __init__(self,input_size, num_experts, hidden_size, output_size):
        super().__init__()

        self.num_experts = num_experts
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.input_size = input_size

        self.experts = nn.ModuleList([
            nn.LSTM(input_size=input_size, hidden_size=hidden_size) for _ in range(num_experts)
        ])

        self.gating_layer = nn.Linear(input_size, num_experts)
        self.Linear1 = nn.Linear(hidden_size, 8)
        self.Linear2 = nn.Linear(8, 4)
        self.Linear3 = nn.Linear(4, 2)
        self.output_layer = nn.Linear(2, output_size)

    def forward(self, x):
        # Get the outputs of the experts.
        expert_outputs = []
        for expert in self.experts:
          exp , (h1,c1) = expert(x)
          #print("each exp", len(exp))
          expert_outputs.append(exp)

        # Get the gating weights.
        expert_outputs = torch.stack(expert_outputs, dim=1)
        gating_weights = self.gating_layer(x)
        print('gating_weights', gating_weights.shape)
        print('expert_outputs',expert_outputs.shape)
        # Compute the weighted sum of the expert outputs.
        weighted_sum = torch.sum(expert_outputs * gating_weights.view(x.shape[0],self.num_experts) , dim=1)
        out = self.Linear1(weighted_sum)
        out = self.Linear2(out)
        out = self.Linear3(out)
        # Output the result.
        return self.output_layer(out)
model = MixtureOfExpertsBard(5,20,32,1)
print(model)

MixtureOfExpertsBard(
  (experts): ModuleList(
    (0-19): 20 x LSTM(5, 32)
  )
  (gating_layer): Linear(in_features=5, out_features=20, bias=True)
  (Linear1): Linear(in_features=32, out_features=8, bias=True)
  (Linear2): Linear(in_features=8, out_features=4, bias=True)
  (Linear3): Linear(in_features=4, out_features=2, bias=True)
  (output_layer): Linear(in_features=2, out_features=1, bias=True)
)


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
LossFuction = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train(dataloader, model, LossFuction, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        #print(X.size())
        pred = model(X)
        loss = torch.sqrt(LossFuction(pred, y))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataLoader, model, LossFuction, optimizer)
print("Done!")

Epoch 1
-------------------------------
gating_weights torch.Size([32, 20])
expert_outputs torch.Size([32, 20, 32])


RuntimeError: ignored